<a href="https://colab.research.google.com/github/moni6264/deeplearning2/blob/main/CNN_PartB_5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PART - B Fine-Tuning a Pre-Trained model**

Import Google colab to drive 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import shutil and splitting the dataset 

In [ ]:
import shutil  # provides the ability to copy and removal of files
import os  # provides functions for interacting with the operating system
import random
try:
    shutil.rmtree("inaturalist_12K")
except:
    pass

# Unzip the natural datset     
zip_path = "drive/MyDrive/nature_12K.zip"
!cp "{zip_path}" .
!unzip -q nature_12K.zip

try:
    os.remove("nature_12K.zip")
except:
    pass
try:
    shutil.rmtree("inaturalist_12K/validation/")
except:
    pass

# dataset directory 
temp = os.listdir("inaturalist_12K/train")
print("Defining class names class names") # print number classes in dataset
class_names = [name for name in temp if name != ".DS_Store"] 

#Spliting valdiation dataset 
validation_split = 0.1 #splitting dataset 10%
def spliting_train_data(validation_split = 0.1):
  os.mkdir("inaturalist_12K/validation/")
  for each_class in class_names:
      os.mkdir("inaturalist_12K/validation/"+each_class)
      train_images = os.listdir("inaturalist_12K/train/"+each_class)
      train_images_per_class = list(train_images)
      print(len(train_images_per_class))
      random.shuffle(train_images_per_class) # Shuffle the training images

      validation_images_per_class = train_images_per_class[:round(validation_split*len(train_images_per_class))]
      print(len(validation_images_per_class))

      for im in validation_images_per_class:
          shutil.move("inaturalist_12K/train/"+each_class+"/"+im, "inaturalist_12K/validation/"+each_class+"/"+im)
spliting_train_data(validation_split)

Defining class names class names
1000
100
1000
100
1000
100
1000
100
1000
100
1000
100
1000
100
1000
100
1000
100
1000
100


Exploring the Data

In [ ]:
train = 0
val = 0
test = 0

for each_class in class_names:
      train_images = os.listdir("inaturalist_12K/train/"+each_class)
      train+=len(list(train_images))

      validation_images = os.listdir("inaturalist_12K/validation/"+each_class)
      
      val+=len(list(validation_images))

      test_images = os.listdir("inaturalist_12K/val/"+each_class)
      
      test+=len(list(test_images))
 # print training validation test      
print("training" , train)
print("validation Examples = ", val)
print("test examples = ", test)

training 9000
validation Examples =  1000
test examples =  2000


Import required libriaries for pre trained module 

In [ ]:
import os  
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout, BatchNormalization


# implement weights and biases 

!pip install wandb
import wandb
from keras.callbacks import EarlyStopping, ModelCheckpoint
from wandb.keras import WandbCallback
from keras.utils.vis_utils import plot_model
from PIL import Image
# image plotting 
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 144 kB 37.9 MB/s 
     |████████████████████████████████| 181 kB 43.9 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=5b2d690cd865fe2733d11d0f314cb3b28fa2b6031ffc0a4e04c3d7fdc691b758
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


Setup the Train Test and Validation generators


In [ ]:
data_dir = "inaturalist_12K"
#data_augmentation = True
# data prepartion 
def data_preparation(data_dir , data_augmentation = True , batch_size = 250):
  train_dir = os.path.join(data_dir, "train")
  val_dir = os.path.join(data_dir, "validation")
  test_dir = os.path.join(data_dir, "val")
  if data_augmentation == True:
    train_datagen = ImageDataGenerator(rescale=1./255,
                                      height_shift_range=0.2,
                                      width_shift_range=0.2,
                                      horizontal_flip=True,
                                      zoom_range=0.2,
                                      fill_mode="nearest")
  else:
      train_datagen = ImageDataGenerator(rescale=1./255)
  val_datagen = ImageDataGenerator(rescale=1./255)
  test_datagen = ImageDataGenerator(rescale=1./255)


  train_generator = train_datagen.flow_from_directory(
          train_dir,
          target_size=(256, 256),
          shuffle=True,
          color_mode="rgb",
          batch_size=batch_size,
          class_mode='categorical')

  val_generator = val_datagen.flow_from_directory(
          val_dir,
          target_size=(256, 256),
          shuffle=True,
          color_mode="rgb",
          batch_size=batch_size,
          class_mode='categorical')
  
  test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size= (256, 256),
        color_mode="rgb",
        batch_size=batch_size,
        class_mode=None)

  
  
  return train_generator , val_generator, test_generator


Implement pre trained InceptionV3, InceptionResNetV2, ResNet50, Xception

In [ ]:
def pretrain_model(pretrained_model_name, dropout, dense_layer, pre_layer_train=None):
    
  # keras sequential model based on a pre-trained model intended to be fine tuned
  

    # shape image size 
    shape = (256 ,256, 3)

    # add a pretrained model without the top dense layer
    if pretrained_model_name == 'ResNet50':
      pretrained_model = tf.keras.applications.ResNet50(include_top=False,input_shape=shape, weights='imagenet')
    elif pretrained_model_name == 'InceptionResNetV2':
      pretrained_model = tf.keras.applications.InceptionResNetV2(include_top=False,input_shape= shape, weights='imagenet')
    elif pretrained_model_name == 'InceptionV3':
      pretrained_model = tf.keras.applications.InceptionV3(include_top=False,input_shape= shape, weights='imagenet')    
    elif pretrained_model_name == 'Xception':
      pretrained_model = tf.keras.applications.Xception(include_top=False,input_shape= shape, weights='imagenet')
    else:
      raise Exception('no pretrained model given')
      
    
    #freeze all layers
    for layer in pretrained_model.layers:
        layer.trainable=False 
    
    #setting  top layers as trainable
    if pre_layer_train:
      for layer in pretrained_model.layers[-pre_layer_train:]:
        layer.trainable=True

    model = tf.keras.models.Sequential()
    model.add(pretrained_model)  #adding pretrained model
    model.add(Flatten()) # flatten layer
    model.add(Dense(dense_layer, activation= 'relu'))#adding a dense layer
    model.add(Dropout(dropout)) # dropout
    model.add(Dense(10, activation="softmax"))#softmax layer
# sum model 
    return model

Configuration wandb


In [ ]:
pre_train_model = "InceptionV3"   #you can change model for wandb sweeps
def train():
    # Default values for hyper-parameters
    config_defaults = {
        "data_augmentation": True,
        "batch_size": 250,
        "dropout": 0.1,
        "dense_layer": 256,
        "learning_rate": 0.0001,
        "epochs": 5,
        "pre_layer_train": None,
        "pretrained_model_name": 'InceptionV3'
      

    }

    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    # Config is a variable that holds and saves hyperparameters 
    config = wandb.config

    # Local variables obtained from wandb config
    data_augmentation = config.data_augmentation
    batch_size = config.batch_size
    dropout = config.dropout
    dense_layer = config.dense_layer
    learning_rate = config.learning_rate
    epochs = config.epochs
    pre_layer_train = config.pre_layer_train
    pretrained_model_name = config.pretrained_model_name
    

    
   # show  the hyperparameters
    run_name = "pre_train_mdl_{}_aug_{}_bs_{}_ep_{}_dropout_{}_dense_{}".format(pre_train_model, data_augmentation, batch_size,epochs, dropout, dense_layer)
    print(run_name)

    #data generators
    train_generator , val_generator, test_generator = data_preparation(data_dir , data_augmentation = True , batch_size = 250)
    
    # calling the pretrained model 
    model = pretrain_model(pretrained_model_name = pretrained_model_name, dropout = dropout, dense_layer = dense_layer, pre_layer_train=pre_layer_train)
    print(model.count_params())

    # compile the model using optmizer  (adam)
    model.compile(optimizer=Adam(learning_rate = learning_rate), loss = 'categorical_crossentropy', metrics = ['accuracy'])

    # Early Stopping method 
    earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')

    # save the best validation accuracy
    checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)

# model fit 
    history = model.fit(train_generator,
                        steps_per_epoch = train_generator.n//train_generator.batch_size,
                        validation_data = val_generator,
                        validation_steps = val_generator.n//val_generator.batch_size,
                        epochs=epochs, verbose = 2, 
                        callbacks=[WandbCallback(), earlyStopping, checkpoint])
      
    

    wandb.run.name = run_name
    wandb.run.save()
    wandb.run.finish()
    return history

sweeps configuration for hyper parameters 

In [ ]:

sweep_config = {
  "name": "DL_Assignment_PartB",
  "metric": {
      "name":"val_accuracy",
      "goal": "maximize"
  },
  "method": "bayes",
  "parameters": {
        "data_augmentation": {
            "values": [True, False]
        },
        "batch_size": {
            "values": [128, 256, 512]
        },
        "learning_rate": {
            "values": [0.001, 0.0001]
        },
        "epochs": {
            "values": [5, 10]
        },
        "dropout": {
            "values": [0, 0.2,0.4]
        },
        "dense_layer": {
            "values": [128, 256, 512]
        },
          "pre_layer_train": {
            "values": [None, 10, 20]
        },
          "pretrained_model_name":{
             "values": ['InceptionResNetV2','ResNet50','InceptionV3','Xception'] 
          },

      
           }
}

sweep_id = "q832m8jh"
wandb.agent(sweep_id,train,entity="moni6264",project="test_part_b",count=5)
#sweep_id = wandb.sweep(sweep_config, entity="moni6264", project="test_part_b")
#wandb.agent(sweep_id, train, count=8)

wandb: Agent Starting Run: xogy3rfr with config:
wandb: 	batch_size: 256
wandb: 	data_augmentation: True
wandb: 	dense_layer: 512
wandb: 	dropout: 0.4
wandb: 	epochs: 5
wandb: 	learning_rate: 0.001
wandb: 	pre_layer_train: 20
wandb: 	pretrained_model_name: Xception


pre_train_mdl_InceptionV3_aug_True_bs_256_ep_5_dropout_0.4_dense_512
Found 8999 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
87975986
Epoch 1/5
35/35 - 301s - loss: 2.9440 - accuracy: 0.3063 - val_loss: 3.2288 - val_accuracy: 0.3240 - _timestamp: 1648233418.0000 - _runtime: 310.0000 - 301s/epoch - 9s/step
Epoch 2/5
35/35 - 293s - loss: 1.2446 - accuracy: 0.5866 - val_loss: 1.3248 - val_accuracy: 0.6160 - _timestamp: 1648233713.0000 - _runtime: 605.0000 - 293s/epoch - 8s/step
Epoch 3/5
35/35 - 286s - loss: 0.8360 - accuracy: 0.7251 - val_loss: 0.9266 - val_accuracy: 0.7390 - _timestamp: 1648233999.0000 - _runtime: 891.0000 - 286s/epoch - 8s/step
Epoch 4/5
35/35 - 288s - loss: 0.6884 - accuracy: 0.7739 - val_loss: 0.7931 - val_accuracy: 0.7510 - _timestamp: 1648234319.0000 - _runtime: 1211.0000 - 288s/epoch - 8s/step
Epoch 5/5
35/35 - 279s - loss: 0.5844 - accuracy: 0.8096 - val_loss: 0.8040 - val_accuracy: 0.7370 -

accuracy,▁▅▇██
epoch,▁▃▅▆█
loss,█▃▂▁▁
val_accuracy,▁▆███
val_loss,█▃▁▁▁
accuracy,0.80958
best_epoch,3
best_val_loss,0.79311
epoch,4
loss,0.5844
val_accuracy,0.737


wandb: Agent Starting Run: xs9hl2ug with config:
wandb: 	batch_size: 256
wandb: 	data_augmentation: True
wandb: 	dense_layer: 512
wandb: 	dropout: 0.4
wandb: 	epochs: 5
wandb: 	learning_rate: 0.0001
wandb: 	pre_layer_train: 20
wandb: 	pretrained_model_name: Xception


pre_train_mdl_InceptionV3_aug_True_bs_256_ep_5_dropout_0.4_dense_512
Found 8999 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
87975986
Epoch 1/5
35/35 - 300s - loss: 1.3875 - accuracy: 0.5764 - val_loss: 0.9415 - val_accuracy: 0.7320 - _timestamp: 1648235020.0000 - _runtime: 309.0000 - 300s/epoch - 9s/step
Epoch 2/5
35/35 - 290s - loss: 0.8114 - accuracy: 0.7370 - val_loss: 0.8751 - val_accuracy: 0.7410 - _timestamp: 1648235338.0000 - _runtime: 627.0000 - 290s/epoch - 8s/step
Epoch 3/5
35/35 - 294s - loss: 0.6806 - accuracy: 0.7783 - val_loss: 0.8027 - val_accuracy: 0.7690 - _timestamp: 1648235659.0000 - _runtime: 948.0000 - 294s/epoch - 8s/step
Epoch 4/5
35/35 - 294s - loss: 0.5951 - accuracy: 0.7988 - val_loss: 0.7773 - val_accuracy: 0.7740 - _timestamp: 1648235958.0000 - _runtime: 1247.0000 - 294s/epoch - 8s/step
Epoch 5/5
35/35 - 278s - loss: 0.5206 - accuracy: 0.8225 - val_loss: 0.8216 - val_accuracy: 0.7550 -

...

wandb: Agent Starting Run: aqv22xrz with config:
wandb: 	batch_size: 128
wandb: 	data_augmentation: True
wandb: 	dense_layer: 512
wandb: 	dropout: 0.4
wandb: 	epochs: 5
wandb: 	learning_rate: 0.0001
wandb: 	pre_layer_train: 20
wandb: 	pretrained_model_name: InceptionResNetV2


pre_train_mdl_InceptionV3_aug_True_bs_128_ep_5_dropout_0.4_dense_512
Found 8999 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
82653930
Epoch 1/5
35/35 - 313s - loss: 1.2167 - accuracy: 0.6486 - val_loss: 0.7989 - val_accuracy: 0.7640 - _timestamp: 1648236643.0000 - _runtime: 331.0000 - 313s/epoch - 9s/step
Epoch 2/5
35/35 - 288s - loss: 0.7585 - accuracy: 0.7593 - val_loss: 0.7023 - val_accuracy: 0.7730 - _timestamp: 1648236932.0000 - _runtime: 620.0000 - 288s/epoch - 8s/step
Epoch 3/5
35/35 - 285s - loss: 0.6858 - accuracy: 0.7797 - val_loss: 0.6690 - val_accuracy: 0.7900 - _timestamp: 1648237216.0000 - _runtime: 904.0000 - 285s/epoch - 8s/step
Epoch 4/5
35/35 - 275s - loss: 0.6308 - accuracy: 0.7919 - val_loss: 0.6777 - val_accuracy: 0.7800 - _timestamp: 1648237498.0000 - _runtime: 1186.0000 - 275s/epoch - 8s/step
Epoch 5/5
35/35 - 273s - loss: 0.6007 - accuracy: 0.8021 - val_loss: 0.6880 - val_accuracy: 0.7850 -

accuracy,▁▆▇██
epoch,▁▃▅▆█
loss,█▃▂▁▁
val_accuracy,▁▃█▅▇
val_loss,█▃▁▁▂
accuracy,0.80215
best_epoch,2
best_val_loss,0.66904
epoch,4
loss,0.60067
val_accuracy,0.785


wandb: Agent Starting Run: zpr2aiww with config:
wandb: 	batch_size: 256
wandb: 	data_augmentation: True
wandb: 	dense_layer: 512
wandb: 	dropout: 0.4
wandb: 	epochs: 5
wandb: 	learning_rate: 0.001
wandb: 	pre_layer_train: 20
wandb: 	pretrained_model_name: Xception


pre_train_mdl_InceptionV3_aug_True_bs_256_ep_5_dropout_0.4_dense_512
Found 8999 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
87975986
Epoch 1/5
35/35 - 289s - loss: 2.5372 - accuracy: 0.3208 - val_loss: 2.6667 - val_accuracy: 0.4970 - _timestamp: 1648238094.0000 - _runtime: 297.0000 - 289s/epoch - 8s/step
Epoch 2/5
35/35 - 285s - loss: 1.2106 - accuracy: 0.6161 - val_loss: 1.4475 - val_accuracy: 0.5340 - _timestamp: 1648238383.0000 - _runtime: 586.0000 - 285s/epoch - 8s/step
Epoch 3/5
